In [1]:
using JuMP
using Gurobi
using Random

In [2]:
N=6 #No. of nodes (1 supplier + 5 DC's)

K=3 #No. of vehicles

Dk=900 #Capacity of vehicles

T=8 #Periods

h=0.025 #Holding cost/unit at each DC

C=[5000 502 488 220 486 742] #Capacity limit for (1 supplier + 5 DC's)

d=[0 0 0 0 0 0 0 0;
75.9 75.9 75.9 75.9 75.9 85.7 85.7 75.9;
62.9 94.8 94.8 94.8 94.8 68.65 36.75 62.9;
67.2 67.2 67.2 67.2 67.2 67.15 67.15 67.2;
102.3 156 156 156 156 119.05 65.35 102.3;
107.4 130.1 130.1 130.1 130.1 65.15 42.45 107.4
] #Demand for each day and (1 supplier + 5 DC's)

c=[
0 140 434 389 419 125;
140 0 300 455 400 97;
434 300 0 609 417 330;
389 455 609 0 256 358;
419 400 417 256 0 316;
125 97 330 358 316 0
] #Transportation cost


I0=[0 98.8 153.9 42.4 23.4 85.7] #Initial inventory


1×6 Matrix{Float64}:
 0.0  98.8  153.9  42.4  23.4  85.7

In [3]:
model=Model(Gurobi.Optimizer)
@variable(model, x[i=1:N,j=1:N,k=1:K,t=1:T], Bin); #If truck k leaves DC i and goes to DC j
@variable(model, y[i=1:N,k=1:K,t=1:T], Bin); #If truck k arrives at DC i
@variable(model, z[j=1:N,k=1:K,t=1:T] >= 0); #Load of truck k when arriving at node j
@variable(model, I[i=1:N,t=0:T] >= 0); #Inventory level at node i till end of period t
@variable(model, q[i=1:N,k=1:K,t=1:T] >= 0); #Quantity delivered to customer i by truck k at start of period t

@objective(model, Min, sum(h*I[i,t] for i=2:N, t=1:T) + sum(c[i,j]*x[i,j,k,t]  for i=1:N, j=1:N, k=1:K, t=1:T))

@constraint(model, [i=2:N], I[i,0] == I0[i]) #Initializing inventory level

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) == I[i,t] + d[i,t]); #Balance constraint for each DC

@constraint(model, [i=2:N,t=1:T], I[i,t-1] + sum(q[i,k,t] for k=1:K) <= C[i]); #Capacity constraint for each DC

@constraint(model, [i=2:N,k=1:K,t=1:T], q[i,k,t] <= C[i]*y[i,k,t]); #Constraint which ensures y is 1 when they receive a quantity

@constraint(model, [k=1:K,t=1:T], sum(q[i,k,t] for i=2:N) <= Dk*y[1,k,t]); #If truck delivers from supplier, it has to activate the supplier

@constraint(model, [a=1:N,k=1:K,t=1:T], sum(i == a ? 0 : x[i,a,k,t] for i = 1:N) == y[a,k,t]); #If truck visits DC

@constraint(model, [a=1:N,k=1:K,t=1:T], sum(j == a ? 0 : x[a,j,k,t] for j = 1:N) == y[a,k,t]); #If truck leaves DC

@constraint(model, [i=2:N,j=2:N,k=1:K,t=1:T], z[i,k,t] - d[i,t] >= z[j,k,t] - (1-x[i,j,k,t])*sum(d)); #To avoid subtours automatically

Academic license - for non-commercial use only - expires 2023-03-02


In [4]:
optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1117 rows, 1350 columns and 4037 nonzeros
Model fingerprint: 0xded68cbd
Variable types: 342 continuous, 1008 integer (1008 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [3e-02, 6e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+03]
Presolve removed 160 rows and 213 columns
Presolve time: 0.02s
Presolved: 957 rows, 1137 columns, 3764 nonzeros
Variable types: 273 continuous, 864 integer (864 binary)

Root relaxation: objective 2.649403e+03, 637 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2649.40295    0   84          - 2649.40295      -     -    0s
H    0     0                    12298.128750 2649.40295  78.5%     -    0s
H    0     0  

In [5]:
if termination_status(model) == MOI.OPTIMAL
   println("RESULTS:")
   println("Objective = $(objective_value(model))")
else
   println("No solution")
end

println();
    for k=1:K
        has_route = false;
        print("Truck:$(value(k))\n")
        for t=1:T
            print("Day:$(value(t))\n")
            for i=1:N
                for j=1:N
                    if (value(x[i,j,k,t]) == 1)
                        print(j,"->",i,"\n")
                        print("$(value(q[i,k,t]))\n")
                        has_route = true;
                    end
                end
            end
        end
        if has_route
            println();
        end
    end

for i=1:N
    for k=1:K
        for t=1:T
            if (value(q[i,k,t]) > 0)
                println(" Node ",i," Truck ",k," Day ", t," Delivery ",value(q[i,k,t]))
            end
        end
    end
end


for t=1:T
    println("Day", t,)
    for i=2:N
        println("Node ",i," Inventory Level = ",value(I[i,t]))
    end
end


RESULTS:


Objective = 4481.57



Truck:1


Day:1
6->1


0.0
1->6
530.5000000000015
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8

Truck:2
Day:1
Day:2
Day:3
4->1
0.0
5->4
201.60000000000002
6->5
471.8
1->6
226.59999999999997
Day:4
Day:5
Day:6
2->1
0.0
3->2
247.3
5->3
168.3
1->4
201.50000000000003
4->5
282.9
Day:7
Day:8

Truck:3
Day:1
2->1
0.0
3->2
280.70000000000005
5->3
288.2
1->

4
91.9999999999995
4->5
234.9
Day:2
Day:3
Day:4
Day:5
Day:6
Day:7
Day:8



 Node 2 Truck 2 Day 6

 Delivery 247.3
 Node 2 Truck 3 Day 1 Delivery 280.70000000000005
 Node 3 Truck 2 Day 6 Delivery 168.3
 Node 3 Truck 3 Day 1 Delivery 288.2
 Node 4 Truck 2 Day 3 Delivery 201.60000000000002
 Node 4 Truck 2 Day 6 Delivery 201.50000000000003
 Node 4 Truck 3 Day 1 Delivery 91.9999999999995
 Node 5 Truck 2 Day 3 Delivery 471.8
 Node 

5 Truck 2 Day 6 Delivery 282.9
 Node 5 Truck 3 Day 1 Delivery 234.9
 Node 6 Truck 1 Day 1 Delivery 530.5000000000015
 Node 6 Truck 2 Day 3 Delivery 226.59999999999997


Day1
Node 2 Inventory Level = 303.6
Node 3 Inventory Level = 379.2
Node 4 Inventory Level = 67.1999999999995
Node 5 Inventory Level = 156.0
Node 6 Inventory Level = 508.80000000000007
Day2
Node 2 Inventory Level = 227.70000000000002
Node 3 Inventory Level = 284.4
Node 4 Inventory Level = 0.0
Node 5 Inventory Level = 0.0
Node 6 Inventory Level = 378.70000000000005
Day3
Node 2 Inventory Level = 151.8
Node 3 Inventory Level = 189.6
Node 4 Inventory Level = 134.4
Node 5 Inventory Level = 315.8
Node 6 Inventory Level = 475.20000000000005
Day4
Node 2 Inventory Level = 75.9
Node 3 Inventory Level = 94.8
Node 4 Inventory Level = 67.2
Node 5 Inventory Level = 159.8
Node 6 Inventory Level = 345.1
Day5
Node 2 Inventory Level = 0.0
Node 3 Inventory Level = 0.0
Node 4 Inventory Level = 0.0
Node 5 Inventory Level = 3.8000000000000114
Node 6 Inventory Level = 215.0
Day6
Node 2 Inventory Level = 161.60000000000002
Node 3 Inventory Level = 99.65
Node 4 Inventory Level = 134.35000000000002
Node 5 Inven